In [86]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end

Running ML-IMC in the training mode.
Building a model...
Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false))
   Number of layers: 4 
   Number of neurons in each layer: [20, 20, 20, 1]


(Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false)), AMSGrad(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.005, 1.006, 1.005, 1.006, 1.006, 1.007, 1.007, 1.007, 1.007], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.003, 1.003, 1.003, 1.004, 1.004, 1.004, 1.005, 1.005, 1.004, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.004, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.005, 1.004, 1.006, 1.003, 1.004, 1.004, 1.005, 1.004]])

In [95]:
systemParms = systemParmsList[1];

In [96]:
traj = readXTC(systemParms)
frame = read_step(traj, 1)
coordinates = positions(frame);

In [97]:
distanceMatrix = buildDistanceMatrix(frame);

In [98]:
cutoff = 6.0 # Å
rs = 0.0 # Å 
eta = 0.1 # Å^-2
lambda = 1.0
zeta = 1.0;

In [99]:
distanceVector = distanceMatrix[:, 1];

In [100]:
@btime $distanceMatrix[:, 1];

  438.170 ns (1 allocation: 4.12 KiB)


In [ ]:
@btime positions($frame)[:, 3];

  49.356 ns (3 allocations: 160 bytes)


In [ ]:
@btime positions($frame)[:, 3] .- positions($frame)[:, 4];

  116.962 ns (7 allocations: 400 bytes)


In [ ]:
coordinates = positions(frame);

In [ ]:
@btime coordinates = positions($frame);

  33.565 ns (2 allocations: 80 bytes)


In [ ]:
@btime $coordinates[:, 5];

  19.239 ns (1 allocation: 80 bytes)


In [ ]:
@btime distance($frame, 5, 6);

  14.856 ns (0 allocations: 0 bytes)


In [126]:
function computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)::Float64
    @assert i != j && i != k && k != j 
    vector_0i = coordinates[:, i]
    vector_ij = coordinates[:, j] .- vector_0i
    vector_ik = coordinates[:, k] .- vector_0i
    cosAngle = dot(vector_ij, vector_ik) / (distance_ij * distance_ik)
    return (cosAngle)
end

computeCosAngle (generic function with 1 method)

In [127]:
@btime computeCosAngle($coordinates, 1, 461, 483, distanceVector[461], distanceVector[483])

  137.054 ns (9 allocations: 480 bytes)


0.7169325422695667

In [102]:
function G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2)
                ) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff))
end        

G9 (generic function with 3 methods)

In [103]:
function G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2 +
                (distance_kj - rshift)^2)) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff) *
        distanceCutoff(distance_kj, cutoff))
end        

G3 (generic function with 3 methods)

In [ ]:
@btime length($distanceVector);

  1.674 ns (0 allocations: 0 bytes)


In [193]:
function G9total(i, coordinates, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    sum = 0.0
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)
    @inbounds for k in eachindex(distanceVector)
        distance_ik = distanceVector[k]
        @inbounds for j in 1:k-1
            distance_ij = distanceVector[j]
            if 0 < distance_ij < cutoff && 0 < distance_ik < cutoff
                cosAngle = computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)
                sum += G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda, rshift)
            end
        end
    end
    return (2.0^(1.0 - zeta) * sum)
end

G9total (generic function with 6 methods)

In [194]:
@btime G9total(5, $coordinates, $distanceMatrix, cutoff, eta, 1.0, 1.0)

  11.546 μs (333 allocations: 29.95 KiB)


7.883061249686378

In [192]:
G9total(5, coordinates, distanceMatrix, cutoff, eta, 1.0, 1.0)

0.6491223630240601

I will implement the G3total function later (because an additional distance has to be computed)